In [2]:
import os
import re
import sys
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import RNN, GRU, LSTM, Dense, Input, Embedding, Dropout, Activation, concatenate
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers

In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
embedding_file = 'glove.6B.300d.txt'

In [5]:
train_data.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
test_data.describe()

,id,comment_text
count,153164,153164
unique,153164,153164
top,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
freq,1,1


In [7]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
train_data.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [10]:
test_data.isnull().any()

id              False
comment_text    False
dtype: bool

In [11]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train_data[classes].values

In [12]:
train_sentences = train_data["comment_text"].fillna("fillna").str.lower()
test_sentences = test_data["comment_text"].fillna("fillna").str.lower()

In [13]:
max_features = 100000
max_len = 150
embed_size = 300

In [14]:
tokenizer = Tokenizer(max_features)
tokenizer.fit_on_texts(list(train_sentences))

In [15]:
tokenized_train_sentences = tokenizer.texts_to_sequences(train_sentences)
tokenized_test_sentences = tokenizer.texts_to_sequences(test_sentences)

In [16]:
train_sentences[1]

"d'aww! he matches this background colour i'm seemingly stuck with. thanks.  (talk) 21:51, january 11, 2016 (utc)"

In [17]:
tokenized_train_sentences[1]

[96145,
 52,
 2635,
 13,
 555,
 3809,
 73,
 4556,
 2706,
 21,
 94,
 38,
 803,
 2679,
 992,
 589,
 8377,
 182]

In [18]:
train_sentences[5]

'"\n\ncongratulations from me as well, use the tools well. \xa0· talk "'

In [19]:
tokenized_train_sentences[5]

[2671, 31, 35, 17, 100, 81, 1, 2228, 100, 13229, 38]

In [20]:
train_padding = pad_sequences(tokenized_train_sentences, max_len)
test_padding = pad_sequences(tokenized_test_sentences, max_len)

In [23]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
try:
    with open(embedding_file, 'r', encoding='utf-8', errors='ignore') as f:
        embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in f)
except FileNotFoundError:
    print(f"File not found: {embedding_file}")
    embeddings_index = {}

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))

for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [24]:
image_input = Input(shape=(max_len, ))
X = Embedding(max_features, embed_size, weights=[embedding_matrix])(image_input)
X = Bidirectional(GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(X)
avg_pl = GlobalAveragePooling1D()(X)
max_pl = GlobalMaxPooling1D()(X)
conc = concatenate([avg_pl, max_pl])
X = Dense(6, activation="sigmoid")(conc)
model = Model(inputs=image_input, outputs=X)

In [25]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
saved_model = "weights_base.best.keras"
checkpoint = ModelCheckpoint(saved_model, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
callbacks_list = [checkpoint, early]

In [30]:
batch_sz = 32
epoch = 2
model.fit(train_padding, y, batch_size=batch_sz, epochs=epoch, validation_split=0.1, callbacks=callbacks_list)

Epoch 1/2
4488/4488 ━━━━━━━━━━━━━━━━━━━━ 2519s 560ms/step - accuracy: 0.8790 - loss: 352339296.0000 - val_accuracy: 0.7822 - val_loss: 0.1221
Epoch 2/2


c:\Users\yasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
c:\Users\yasha\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


4488/4488 ━━━━━━━━━━━━━━━━━━━━ 2110s 470ms/step - accuracy: 0.8764 - loss: 324461952.0000 - val_accuracy: 0.1652 - val_loss: 0.1276


In [31]:
test_values = model.predict([test_padding], batch_size=1024, verbose=1)

150/150 ━━━━━━━━━━━━━━━━━━━━ 99s 645ms/step


In [32]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission[classes] = test_values
sample_submission.to_csv('submission.csv', index=False)

In [33]:
from sklearn.metrics import accuracy_score
from keras.callbacks import Callback

In [34]:
class PrintTestAccuracy(Callback):
    def __init__(self, test_data, test_padding):
        self.test_data = test_data
        self.test_padding = test_padding

    def on_epoch_end(self, epoch, logs=None):
        test_values = self.model.predict([self.test_padding], batch_size=1024, verbose=0)
        test_labels = self.test_data[classes].values
        accuracy = []
        for i, class_name in enumerate(classes):
            y_true = test_labels[:, i]
            y_pred = test_values[:, i]
            accuracy.append(accuracy_score(y_true, y_pred > 0.5))
        print(f"Epoch {epoch+1}/{epoch} - Test Accuracy: {', '.join([f'{c}: {a:.4f}' for c, a in zip(classes, accuracy)])}")

In [35]:
test_accuracy = PrintTestAccuracy(test_data, test_padding)
callbacks_list = [checkpoint, early, test_accuracy]

In [40]:
def preprocess_input(text):
    text = [text.lower()]
    sequences = tokenizer.texts_to_sequences(text)
    padded_sequence = pad_sequences(sequences, maxlen=max_len)
    return padded_sequence

In [42]:
print("Enter a comment for manual testing (or 'q' to quit):")

while True:
    user_input = input("> ")
    if user_input.lower() == 'q':
        break

    padded_input = preprocess_input(user_input)
    prediction = model.predict(padded_input)

    print("Predicted labels:")
    for i, class_name in enumerate(classes):
        label = 1 if prediction[0][i] >= 0.5 else 0
        print(f"{class_name}: {label}")

    print("\nEnter the true labels (0 or 1) for each class (or 'q' to skip):")
    true_labels = []
    for class_name in classes:
        label = input(f"{class_name}: ")
        if label.lower() == 'q':
            print("Skipping accuracy calculation.\n")
            break
        try:
            true_labels.append(int(label))
        except ValueError:
            print(f"Invalid input for {class_name}. Please enter 0 or 1.\n")
            break
    else:
        accuracy = sum(pred == true for pred, true in zip(prediction[0], true_labels)) / len(classes)
        print(f"\nAccuracy: {accuracy:.4f}\n")

Enter a comment for manual testing (or 'q' to quit):
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted labels:
toxic: 0
severe_toxic: 0
obscene: 0
threat: 0
insult: 0
identity_hate: 0

Enter the true labels (0 or 1) for each class (or 'q' to skip):
Invalid input for toxic. Please enter 0 or 1.

